In [1]:
import pymysql as sql
import pandas as pd
import numpy as np
import names
import datetime
import random
from faker import Faker
fake = Faker()

In [2]:
User_data = pd.DataFrame()

# 链接DB

In [3]:
db = sql.connect(host='34.130.109.226',
                             user='root',
                             password='123456',
                             db='DB_PROJ')
db.ping() #只要不报错就说明连上了

In [4]:
cursor = db.cursor()

# Customers

## 生成生日

In [5]:
start_date = datetime.date(1920, 1, 1)
end_date = datetime.date(2011, 1, 1)

time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days
random_number_of_days = random.randrange(days_between_dates)

In [6]:
User_data["DOB"]=[(start_date + datetime.timedelta(days=random.randrange(days_between_dates))) for i in range(200)]

## 生成性别

In [7]:
User_data["Gender"] = [np.random.choice(["M", "F", "U"], p=[0.45, 0.45, 0.1]) for i in range(200)]

## 生成电话号码

In [8]:
User_data["Phone_Num"]=[random.randint(1000000000, 9999999999) for i in range(200)]

## 生成姓名

In [9]:
User_data["F_name"]=[names.get_first_name() for i in range(200)]
User_data["L_name"]=[names.get_last_name() for i in range(200)]

## 生成邮箱

In [10]:
User_data["Email"]=User_data["F_name"]+User_data["L_name"]+np.array([str(random.randint(0, 10)) for i in range(200)])+"@sample.com"

In [11]:
User_data=User_data.reset_index().rename(columns={"index":"Customer_id"})
User_data["Customer_id"]+=1
User_data

Customer_id         DOB Gender   Phone_Num       F_name     L_name  \
0              1  1982-01-08      U  7857598318       Thomas    Pierson   
1              2  2001-11-11      U  8691238144      Dorothy      Jones   
2              3  1998-09-27      M  5365417127      Bridget      Suter   
3              4  1930-12-03      F  1163188313       Bonnie     Powell   
4              5  1973-07-26      F  2457467836      Marlene      Symes   
..           ...         ...    ...         ...          ...        ...   
195          196  1961-09-01      M  3981190327        Hazel      Aaron   
196          197  1923-08-12      M  2760568694       Arthur      Haydu   
197          198  2006-05-02      M  8511255787      Heather       Urie   
198          199  1990-10-09      M  7389987670       Robert  Hettinger   
199          200  1994-11-24      M  3952152384  Christopher       Nash   

                            Email  
0       ThomasPierson3@sample.com  
1       DorothyJones10@sample.com  
2        BridgetSuter7@sample.com  
3        BonniePowell7@sample.com  
4        MarleneSymes9@sample.com  
..                            ...  
195        HazelAaron9@sample.com  
196       ArthurHaydu1@sample.com  
197       HeatherUrie9@sample.com  
198  RobertHettinger10@sample.com  
199   ChristopherNash5@sample.com  

[200 rows x 7 columns]

In [12]:
for i, row in User_data.iterrows():
    s = f'''INSERT INTO `Customers`(`Customer_id`, `DOB`, `Gender`, `Phone_Num`, `F_name`, `L_name`,`Email`)VALUES({row["Customer_id"]}, '{str(row["DOB"])}', '{row["Gender"]}', '{row["Phone_Num"]}', '{row["F_name"]}', '{row["L_name"]}', '{row["Email"]}')'''
    cursor.execute(s)

In [13]:
db.commit()

# 生成住址

In [14]:
Customer_Address=pd.DataFrame()

In [15]:
Customer_Address["Customer_id"]=list(range(1, 201))
Customer_Address["State"]=["NY"]*200
Customer_Address["City"]=["Rochester"]*200
Customer_Address["Street"]=[fake.address().split("\n")[0] for i in range(200)]
Customer_Address["Zip_Code"]=[np.random.choice(list(range(14604, 14628))) for i in range(200)]
Customer_Address

Customer_id State       City                          Street  Zip_Code
0              1    NY  Rochester   7837 Desiree Centers Apt. 075     14626
1              2    NY  Rochester              PSC 8145, Box 8464     14624
2              3    NY  Rochester                7482 Aaron Coves     14615
3              4    NY  Rochester    0933 Taylor Freeway Apt. 179     14610
4              5    NY  Rochester                08646 James Land     14606
..           ...   ...        ...                             ...       ...
195          196    NY  Rochester  3648 Erika Stravenue Suite 449     14616
196          197    NY  Rochester      314 Short Terrace Apt. 183     14614
197          198    NY  Rochester              PSC 4266, Box 9124     14612
198          199    NY  Rochester       5914 Dillon Hill Apt. 560     14623
199          200    NY  Rochester              Unit 7842 Box 9425     14627

[200 rows x 5 columns]

In [16]:
for i, row in Customer_Address.iterrows():
    s = f'''INSERT INTO `Customers_Address` VALUES({row["Customer_id"]}, '{str(row["State"])}', '{row["City"]}', '{row["Street"]}', {row["Zip_Code"]})'''
    if i %50==0: print(i)
    cursor.execute(s)

0
50
100
150


In [17]:
db.commit()

# 添加餐馆

In [18]:
Restaurants = pd.DataFrame()

In [19]:
Restaurants["Restaurant_id"] = list(range(1, 21))
Restaurants["Name"] = [names.get_first_name()+"'s"+np.random.choice([" Restaurant", "", " Roadhouse"]) for i in range(20)]
Restaurants["Address"] = [fake.address().split("\n")[0] + " Rochester, NY "+str(np.random.choice(list(range(14604, 14628)))) for i in range(20)]
Restaurants["Phone_Num"] = [random.randint(1000000000, 9999999999) for i in range(20)]
Restaurants["Email"] = Restaurants["Name"].str.replace(" ", "").str.replace("'", "")+"@sample.com"
Restaurants["Website"] = "www."+Restaurants["Name"].str.replace(" ", "").str.replace("'", "")+".com"
Restaurants["Category"] = [np.random.choice(["Chinese", "American", "Japanese", "Korean", "Indian", "Vietnamese"], p=[0.2, 0.45, 0.1, 0.1, 0.1, 0.05]) for i in range(20)]
Restaurants["Name"] = Restaurants["Name"].str.replace("'", r"\'")
Restaurants

Restaurant_id                    Name  \
0               1      Lynn\'s Restaurant   
1               2                James\'s   
2               3                Jamie\'s   
3               4                 Ruby\'s   
4               5  Jermaine\'s Restaurant   
5               6     Yvette\'s Roadhouse   
6               7      Lynn\'s Restaurant   
7               8                Maria\'s   
8               9                Jason\'s   
9              10      John\'s Restaurant   
10             11   Jennifer\'s Roadhouse   
11             12  Jonathon\'s Restaurant   
12             13     Caleb\'s Restaurant   
13             14     Gloria\'s Roadhouse   
14             15                 Gail\'s   
15             16    Judith\'s Restaurant   
16             17    Albert\'s Restaurant   
17             18     Carol\'s Restaurant   
18             19   William\'s Restaurant   
19             20   Zachary\'s Restaurant   

                                              Address   Phone_Num  \
0             64694 Peters Summit Rochester, NY 14623  6107190046   
1        7788 Carl Mall Suite 211 Rochester, NY 14607  7673776700   
2       144 Little Forks Apt. 407 Rochester, NY 14604  4831179525   
3              09449 Cochran Walk Rochester, NY 14621  6022615088   
4     299 Gerald Highway Apt. 752 Rochester, NY 14615  3693522945   
5                      USS Murphy Rochester, NY 14614  8805395483   
6            7870 White Stravenue Rochester, NY 14626  1722132015   
7         82486 Jennings Causeway Rochester, NY 14605  5441348800   
8             188 Copeland Hollow Rochester, NY 14616  7066214719   
9               223 Carrie Greens Rochester, NY 14620  6754850656   
10   145 Willie Village Suite 490 Rochester, NY 14623  6981663027   
11                5592 Alexa Park Rochester, NY 14605  8866043836   
12                04239 Jones Via Rochester, NY 14625  3511586755   
13               9087 Ryan Square Rochester, NY 14625  7835381722   
14    768 Grant Centers Suite 267 Rochester, NY 14617  3045873952   
15        70992 Paul Rue Apt. 936 Rochester, NY 14605  1123039100   
16      842 Martin Vista Apt. 983 Rochester, NY 14607  4031551616   
17       698 Laura Pines Apt. 207 Rochester, NY 14608  9172329985   
18                     USCGC Pham Rochester, NY 14609  5702149534   
19  81216 Charles Vista Suite 789 Rochester, NY 14623  2435378395   

                             Email                      Website  Category  
0       LynnsRestaurant@sample.com      www.LynnsRestaurant.com    Indian  
1                Jamess@sample.com               www.Jamess.com    Indian  
2                Jamies@sample.com               www.Jamies.com  American  
3                 Rubys@sample.com                www.Rubys.com    Korean  
4   JermainesRestaurant@sample.com  www.JermainesRestaurant.com    Indian  
5      YvettesRoadhouse@sample.com     www.YvettesRoadhouse.com   Chinese  
6       LynnsRestaurant@sample.com      www.LynnsRestaurant.com    Korean  
7                Marias@sample.com               www.Marias.com  American  
8                Jasons@sample.com               www.Jasons.com  American  
9       JohnsRestaurant@sample.com      www.JohnsRestaurant.com   Chinese  
10   JennifersRoadhouse@sample.com   www.JennifersRoadhouse.com    Indian  
11  JonathonsRestaurant@sample.com  www.JonathonsRestaurant.com  American  
12     CalebsRestaurant@sample.com     www.CalebsRestaurant.com   Chinese  
13     GloriasRoadhouse@sample.com     www.GloriasRoadhouse.com   Chinese  
14                Gails@sample.com                www.Gails.com  American  
15    JudithsRestaurant@sample.com    www.JudithsRestaurant.com   Chinese  
16    AlbertsRestaurant@sample.com    www.AlbertsRestaurant.com  American  
17     CarolsRestaurant@sample.com     www.CarolsRestaurant.com  Japanese  
18   WilliamsRestaurant@sample.com   www.WilliamsRestaurant.com  American  
19   ZacharysRestaurant@sample.com   www.ZacharysRestaurant.com   Chinese

In [20]:
for i, row in Restaurants.iterrows():
    s = f'''INSERT INTO `Restaurants` VALUES({row["Restaurant_id"]}, '{str(row["Name"])}', '{row["Address"]}', '{row["Phone_Num"]}', '{row["Email"]}', '{row["Website"]}', '{row["Category"]}')'''
    if i %10==0: print(i)
    cursor.execute(s)

0
10


In [21]:
db.commit()

# 生成司机

In [22]:
Drivers = pd.DataFrame()

In [23]:
Drivers["Driver_id"] = list(range(1, 51))
Drivers["F_Name"] = [names.get_first_name() for i in range(50)]
Drivers["L_Name"]=[names.get_last_name() for i in range(50)]
Drivers["SSN"] = [str(random.randint(0, 999999999)).zfill(9) for i in range(50)]
Drivers["Address"] = [fake.address().split("\n")[0] + " Rochester, NY "+str(np.random.choice(list(range(14604, 14628)))) for i in range(50)]
Drivers["Phone_Num"] = [random.randint(1000000000, 9999999999) for i in range(50)]
Drivers["Email"]=Drivers["F_Name"]+Drivers["L_Name"]+np.array([str(random.randint(0, 10)) for i in range(50)])+"@sample.com"
Drivers["Gender"] = [np.random.choice(["M", "F", "U"], p=[0.5, 0.4, 0.1]) for i in range(50)]
Drivers["DOB"]=[(start_date + datetime.timedelta(days=random.randrange(days_between_dates))) for i in range(50)]
Drivers

Driver_id     F_Name      L_Name        SSN  \
0           1   Lawrence      Virgil  512317736   
1           2    Richard      Cannon  978668702   
2           3    Richard    Thompson  876892917   
3           4     Joanne   Cryderman  210957885   
4           5       John    Pichardo  144955431   
5           6    Raymond       Nealy  644090960   
6           7   Margaret      Troyer  594476485   
7           8     Claire     Sampson  936304036   
8           9       Mary     Bernier  048170696   
9          10    Raymond    Franklin  839829720   
10         11      Wanda      Burton  251862473   
11         12         Wm      French  069189731   
12         13  Christine      Laxton  139354290   
13         14    Suzanne      Mcshea  349668323   
14         15    Mellisa     Freeman  483815285   
15         16      Penny       Scott  400972717   
16         17   Margaret      Thalls  781423577   
17         18     Claude      Quayle  606118922   
18         19       Stan       Boone  586435357   
19         20     Ramona   Pritchett  179609814   
20         21      Louis       Kokko  958741518   
21         22      Chong      Holmes  438525251   
22         23      Erika       Silva  854456732   
23         24    Alberto       Fries  771976214   
24         25    Michael      Graves  830087032   
25         26    Barbara     Stewart  005936029   
26         27    Jessica  Brookshire  225803663   
27         28      Tisha        Amyx  735565701   
28         29       Bill       Smith  103431576   
29         30    William      Patton  856678800   
30         31    Gregory       Luthi  877325272   
31         32       Mary       Davis  047241517   
32         33      Paula    Williams  005346541   
33         34      Jacob   Stallings  061932137   
34         35      Lloyd     Chapman  429701922   
35         36      Billy      Sherry  183109595   
36         37      Wanda      Campos  851945053   
37         38      Ralph      Bonato  452436251   
38         39      Elida      Burton  813941025   
39         40    Kenneth    Robinson  405980266   
40         41  Stephanie      Jordan  487781922   
41         42       Lisa        Cada  484562858   
42         43      Henry       Smith  943450785   
43         44    Abraham       Bough  446258026   
44         45    Bradley     Horsley  783168956   
45         46      David     Benford  024697439   
46         47    Vanessa       Smith  344814736   
47         48        Joe      Ramsey  444820561   
48         49     Robert       Kelly  376925262   
49         50      Donna       Kelly  068571308   

                                              Address   Phone_Num  \
0   8140 Cynthia Islands Apt. 436 Rochester, NY 14620  6571828990   
1               75473 White Place Rochester, NY 14608  7343656302   
2     7335 Brian Squares Apt. 729 Rochester, NY 14609  5684257511   
3   553 Williamson Avenue Suite 341 Rochester, NY ...  7469505226   
4      775 Chelsea Roads Apt. 118 Rochester, NY 14621  8506784844   
5         123 Dawn Locks Apt. 764 Rochester, NY 14612  1395333082   
6              Unit 4580 Box 0793 Rochester, NY 14609  1046126800   
7              70899 Ayers Corner Rochester, NY 14617  1131991916   
8    23172 Kaufman Well Suite 894 Rochester, NY 14619  9067904899   
9              PSC 4061, Box 6910 Rochester, NY 14625  3407774428   
10    6722 Robert Cliff Suite 388 Rochester, NY 14605  9646651486   
11   8036 Pacheco Summit Apt. 920 Rochester, NY 14619  3033730352   
12             0941 Molina Plains Rochester, NY 14627  8127485613   
13       755 Tracy Loop Suite 497 Rochester, NY 14622  5278828683   
14  34520 Nicole Harbors Apt. 370 Rochester, NY 14608  7754864461   
15                 532 Reyes Port Rochester, NY 14625  8418327740   
16   9498 Michael River Suite 822 Rochester, NY 14606  7853551916   
17            143 Robert Crossing Rochester, NY 14610  4516403399   
18      7287 Branch Well Apt. 651 Rochester, NY 14621  5154753775   
19  4067 Williams Rive

In [24]:
for i, row in Drivers.iterrows():
    s = f'''INSERT INTO `Drivers` VALUES({row["Driver_id"]}, '{str(row["F_Name"])}', '{row["L_Name"]}', '{row["SSN"]}', '{row["Address"]}', '{row["Phone_Num"]}', '{row["Email"]}', '{row["Gender"]}', '{row["DOB"]}')'''
    if i %10==0: print(i)
    cursor.execute(s)

0
10
20
30
40


In [25]:
db.commit()

# Orders

In [26]:
Orders = pd.DataFrame()

In [27]:
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2021, 11, 15)

time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days
random_number_of_days = random.randrange(days_between_dates)

In [28]:
Orders["Order_id"] = list(range(1, 201))
Orders["Cust_id"] = [np.random.choice(User_data["Customer_id"]) for i in range(200)]
Orders["Driver_id"] = [np.random.choice(Drivers["Driver_id"]) for i in range(200)]
Orders["Restaurant_id"] = [np.random.choice(Restaurants["Restaurant_id"]) for i in range(200)]
Orders["Time"] = [str(start_date + datetime.timedelta(days=random_number_of_days))+" "+str(random.randint(0, 23)).zfill(2)+ ":" +str(random.randint(0, 59)).zfill(2) + ":" + str(random.randint(0,59)).zfill(2) for i in range(200)]
Orders["Software"] = [np.random.choice(["Yelp", "Chowbus", "Grabhub", "Uber Eats", "DoorDash"]) for i in range(200)]
Orders["Delivery_Address"] = Customer_Address["Street"].iloc[Orders["Cust_id"]-1].reset_index(drop=True)+" "+Customer_Address["City"].iloc[Orders["Cust_id"]-1].reset_index(drop=True)+","+Customer_Address["State"].iloc[Orders["Cust_id"]-1].reset_index(drop=True)+" "+Customer_Address["Zip_Code"].iloc[Orders["Cust_id"]-1].reset_index(drop=True).astype(str)
Orders["Payment_Method"] = [np.random.choice(["Card", "Cash", "PayPal"], p=[0.6, 0.25, 0.15]) for i in range(200)]
Orders["Price"] = [round(random.random()*50+20, 2) for i in range(200)]
Orders["Delivery_Instruction"] = [np.random.choice(["Left at Front Door plz", "Hand to me", "Call me when delivered."]) for i in range(200)]

In [29]:
Orders

Order_id  Cust_id  Driver_id  Restaurant_id                 Time  \
0           1      171         23             13  2021-11-13 04:47:15   
1           2        3         48              8  2021-11-13 00:58:43   
2           3      198         32             13  2021-11-13 22:06:04   
3           4      192         15             14  2021-11-13 13:03:55   
4           5       79         25             10  2021-11-13 10:46:58   
..        ...      ...        ...            ...                  ...   
195       196       69          2             11  2021-11-13 17:21:03   
196       197       55         10             10  2021-11-13 12:39:07   
197       198       72         34             12  2021-11-13 22:33:43   
198       199       85         47             14  2021-11-13 04:07:59   
199       200      133         23             17  2021-11-13 02:23:17   

      Software                                   Delivery_Address  \
0      Chowbus                 201 Bruce Place Rochester,NY 14611   
1    Uber Eats                7482 Aaron Coves Rochester,NY 14615   
2    Uber Eats              PSC 4266, Box 9124 Rochester,NY 14612   
3    Uber Eats               8885 Walsh Garden Rochester,NY 14610   
4         Yelp              PSC 9991, Box 4101 Rochester,NY 14617   
..         ...                                                ...   
195   DoorDash                825 David Branch Rochester,NY 14607   
196    Grabhub              PSC 9042, Box 0266 Rochester,NY 14614   
197    Chowbus  43517 Zimmerman River Apt. 133 Rochester,NY 14608   
198    Chowbus     9318 Wright Grove Suite 454 Rochester,NY 14616   
199   DoorDash        718 David Pine Suite 805 Rochester,NY 14620   

    Payment_Method  Price     Delivery_Instruction  
0             Card  53.10               Hand to me  
1           PayPal  25.54  Call me when delivered.  
2             Card  65.02  Call me when delivered.  
3             Cash  43.65  Call me when delivered.  
4             Cash  55.16  Call me when delivered.  
..             ...    ...                      ...  
195           Card  46.26  Call me when delivered.  
196           Card  59.43   Left at Front Door plz  
197           Card  59.04  Call me when delivered.  
198           Cash  51.82  Call me when delivered.  
199           Card  66.16  Call me when delivered.  

[200 rows x 10 columns]

In [30]:
for i, row in Orders.iterrows():
    s = f'''INSERT INTO `Orders` VALUES({row["Order_id"]}, {str(row["Cust_id"])}, {row["Driver_id"]}, {row["Restaurant_id"]}, '{row["Time"]}', '{row["Software"]}', '{row["Delivery_Address"]}', '{row["Payment_Method"]}', {row["Price"]}, '{row["Delivery_Instruction"]}')'''
    if i %50==0: print(i)
    cursor.execute(s)

0
50
100
150


In [31]:
db.commit()

# Delivered_By

In [32]:
Delivered_By = pd.DataFrame()

In [33]:
Delivered_By["Order_id"]=Orders["Order_id"]
Delivered_By["Driver_id"]=[np.random.choice(Drivers["Driver_id"]) for i in range(200)]
Delivered_By["Tips"] = [round(random.random()*5+2, 2) for i in range(200)]
Delivered_By

Order_id  Driver_id  Tips
0           1         47  5.07
1           2         41  2.84
2           3         26  5.63
3           4         33  6.28
4           5         49  5.14
..        ...        ...   ...
195       196         30  6.48
196       197         43  2.66
197       198         25  6.47
198       199         39  4.05
199       200         37  4.50

[200 rows x 3 columns]

In [34]:
for i, row in Delivered_By.iterrows():
    s = f'''INSERT INTO `Delivered_By` VALUES({row["Order_id"]}, {str(row["Driver_id"])}, {row["Tips"]})'''
    if i %50==0: print(i)
    cursor.execute(s)

0
50
100
150


In [35]:
db.commit()

# Comments

In [36]:
Comments = pd.DataFrame()

In [37]:
s='''{"review_id":"lWC-xP3rd6obsecCYsGZRg","user_id":"ak0TdVmGKo4pwqdJSTLwWw","business_id":"buF9druCkbuXLX526sGELQ","stars":4.0,"useful":3,"funny":1,"cool":1,"text":"Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four cheese filled pasta in a creamy sauce with bacon, asparagus and grana frita. Both were very good. We split a secondi which was the special Berkshire pork secreto, which was described as a pork skirt steak with garlic potato purée and romanesco broccoli (incorrectly described as a romanesco sauce). Some tables received bread before the meal but for some reason we did not. \n\nManagement also seems capable for when the tenants in the apartment above began playing basketball she intervened and also comped the tables a dessert. We ordered the apple dumpling with gelato and it was also quite tasty. Portions are not huge which I particularly like because I prefer to order courses. If you are someone who orders just a meal you may leave hungry depending on you appetite. Dining room was mostly younger crowd while the bar was definitely the over 40 set. Would recommend that the naysayers return to see the improvement although I personally don't know the former glory to be able to compare. Easy access to downtown Salem without the crowds on this month of October.","date":"2014-10-11 03:34:02"}
{"review_id":"8bFej1QE5LXp4O05qjGqXA","user_id":"YoVfDbnISlW0f7abNQACIg","business_id":"RA4V8pr014UyUbDvI-LW2A","stars":4.0,"useful":1,"funny":0,"cool":0,"text":"This store is pretty good. Not as great as Walmart (or my preferred, Milford Target), but closer and in a easier area to get to.  \nThe store itself is pretty clean and organized, the staff are friendly (most of the time), and BEST of all is the Self Checkout this store has! \nGreat clearance sections throughout, and great prices on everything in the store, in general (they pricematch too!). \nChristian, Debbie, Jen and Hanna are all very friendly, helpful, sensitive to all customer needs. Definitely one of the better Target locations in the area, and they do a GREAT job assisting customers for being such a busy store. Located directly in the Framingham Mall on Cochituate Rd \/ Route 30. 4 stars.","date":"2015-07-03 20:38:25"}
{"review_id":"NDhkzczKjLshODbqDoNLSg","user_id":"eC5evKn1TWDyHCyQAwguUw","business_id":"_sS2LBIGNT5NQb6PD1Vtjw","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"I called WVM on the recommendation of a couple of friends who had used them in the past and thought they did a nice job. I'm a fan now, too.\n\nEvan and Cody showed up right on time for my move this past weekend. They were friendly and energetic, working quickly but carefully to get all my things moved out of the old place and into the new one in less than 2.5 hours. All of my (heavy) furniture arrived in perfect condition, and they took extra care not to scratch the wood floors in the process.\n\nI hope not to move again anytime soon, but next time I do, I'll be calling WVM.","date":"2013-05-28 20:38:06"}
{"review_id":"T5fAqjjFooT4V0OeZyuk1w","user_id":"SFQ1jcnGguO0LYWnbbftAA","business_id":"0AzLzHfOJgL7ROwhdww2ew","stars":2.0,"useful":1,"funny":1,"cool":1,"text":"I've stayed at many Marriott and Renaissance Marriott's and this was a huge disappointment! The front desk and atrium is nice..there is a starbucks on site which is nice.\n\nThe rooms are run down and old.  There is a flat screen but that is to be expected of a Renaissance.\n\nWe got this hotel via Priceline at a rate of $75\/night...good deal for the price but this is not a true Renaissance.","date":"2010-01-08 02:29:15"}
{"review_id":"sjm_uUcQVxab_EeLCqsYLg","user_id":"0kA0PAJ8QFMeveQWHFqz2A","business_id":"8zehGz9jnxPqXtOc7KaJxA","stars":4.0,"useful":0,"funny":0,"cool":0,"text":"The food is always great here. The service from both the manager as well as the staff is super. Only draw back of this restaurant is it's super loud. If you can, snag a patio table!","date":"2011-07-28 18:05:01"}
{"review_id":"J4a2TuhDasjn2k3wWtHZnQ","user_id":"RNm_RWkcd02Li2mKPRe7Eg","business_id":"xGXzsc-hzam-VArK6eTvtw","stars":1.0,"useful":2,"funny":0,"cool":0,"text":"This place used to be a cool, chill place. Now its a bunch of neanderthal bouncers hopped up on steroids acting like the can do whatever they want. There are so many better places in davis square where they are glad you are visiting their business. Sad that the burren is now the worst place in davis.","date":"2018-01-21 04:41:03"}
{"review_id":"28gGfkLs3igtjVy61lh77Q","user_id":"Q8c91v7luItVB0cMFF_mRA","business_id":"EXOsmAB1s71WePlQk0WZrA","stars":2.0,"useful":0,"funny":0,"cool":0,"text":"The setting is perfectly adequate, and the food comes close.   The dining chains like Chilis and Victoria Station do barbecue better.\n\nIt's no surprise you can always pick up coupons for Linwood at restaurant.com.","date":"2006-04-16 02:58:44"}
{"review_id":"9vqwvFCBG3FBiHGmOHMmiA","user_id":"XGkAG92TQ3MQUKGX9sLUhw","business_id":"DbXHNl890xSXNiyRczLWAg","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"Probably one of the better breakfast sandwiches I've ever had. I had the EGGMEATMUFFIN, the bread was toasted perfectly and the bacon was a real thick cut. Not that lame bacon we are more familiar with at your conventional breakfast diner. In addition, the place was clean and the staff was very helpful. The butcher had several different cuts available and was knowledgable as well as friendly. I left with some cuts of pork and beef and am excited to come back!","date":"2017-12-02 18:16:13"}
{"review_id":"2l_TDrQ7p-5tANOyiOlkLQ","user_id":"LWUnzwK0ILquLLZcHHE1Mw","business_id":"mD-A9KOWADXvfrZfwDs-jw","stars":4.0,"useful":1,"funny":0,"cool":0,"text":"I am definitely a fan of Sports Authority.  This particular location has a good check in deal.  We came here near Christmas time to buy some presents and we had a good experience.  The staff members were very friendly and they helped us find what we were looking for.  We got some golf stuff, two pairs of shoes, a tennis racket bag, and some bicycle accessories.  The store was clean and well organized.  They have everything from apparel to basketballs and everything in between.  Good spot to buy athletic merchandise.","date":"2012-05-28 15:00:47"}
{"review_id":"KKVFopqzcVfcubIBxmIjVA","user_id":"99RsBrARhhx60UnAC4yDoA","business_id":"EEHhKSxUvJkoPSzeGKkpVg","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"I work in the Pru and this is the most affordable and tasty place in the food court.  deals where a meal is $5-$7 and the chicken pesto is really good.   I am not a chowda person but all there soups I have had are pretty damn good.  Broccoli chicken is my favorite.  Also, probably the most personable Food court staff I have ever had the pleasure of ordering from.","date":"2014-05-07 18:10:21"}
{"review_id":"FdoBFTjXXMn4hVnJ59EtiQ","user_id":"eLAYHxHUutiXswy-CfeiUw","business_id":"WQFn1A7-UAA4JT5YWiop_w","stars":1.0,"useful":0,"funny":0,"cool":0,"text":"They NEVER seem to get our \norder correct, service is crappy, food is inconsistent and has gone down hill steadily in the last 6-9 months! WILL NEVER GO THERE AGAIN!","date":"2017-09-08 23:26:10"}
{"review_id":"Qv_lQb1Yu9DGsUO3r-enNw","user_id":"Ngl83gs3n22SzLAsNw2znw","business_id":"iZxfWlfNPXV_0S1xO8QNSQ","stars":2.0,"useful":3,"funny":0,"cool":0,"text":"Disappointing. Bolt is a follow up of Fanny's Fabrics, which closed. Like Fanny's, the selection is small. The fabrics lean toward contemporary. They also have a small stock of fabrics that you can order; however, they don't have enough room to display their selections, so it's like trying to paw through your closet when it has twice as many clothes as the racks were meant to handle. \n\nThe woman who works there most of the time is very nice and helpful. However, my mother and I both bought fabric there, and we both had the same problem: the employee helped us calculate our yardage without taking the repeat into consideration. I brought my pillow forms with me, so there was no doubt exactly how large they were. My mother was able to make her project work, but I came up a pillow short. By the time the pillows were made, I realized I was short, and I returned to Bolt (all in a month's time), the fabric had sold out, with no plans on reordering. \n\nFortunately for me, I was just making a number of large throw pillows. Now I don't have the number I wanted, but I can be flexible. What if this had been an upholstery job and my piece was partially completed? Obviously, unacceptable. My advise is to feel free and shop here, but figure out your measurements on your own.","date":"2013-01-02 18:12:15"}
{"review_id":"Gi5LSRmTXoL9Bp4jNGPjLw","user_id":"hn0ZbitvmlHnF--KJGJ6_A","business_id":"TA1KUSCu8GkWP9w0rmElxw","stars":4.0,"useful":0,"funny":0,"cool":0,"text":"I have been here twice and have had really good food and service both times. I got a burger with caramelized onions, a red wine sauce and I believe blue cheese the first time. It was excellent and just the right size for me. I hate going out and ordering a burger that is bigger than my head that I can't even finish half of! Flip's burgers are about the same size I would make for myself at home.\n\nThe second time I went I had a limited time seasonal offering which was a venison burger with pea shoots, some kind of cheese that reminded me of brie, and a very sweet syrupy sauce which I could have done without, but overall it was very good. I also had onion rings and they were possibly the best onion rings I've had ever.\n\nThe only thing I don't really like about Flip is the atmosphere and layout of the tables. I guess they were going for trendy upscale or whatever, but it makes me feel as if I am in a school cafeteria back in high school. I don't like it. There are long white tables shoved up against each other with uncomfortable modern looking plastic chairs that kind of look like they came from Ikea. It's also pretty loud in there because there is nothing to absorb the sound, like a school cafeteria.\n\nThe last time I was in there there were a couple of women with small children. I have no idea why someone would bring a child to Flip unless that child was a very adventerous eater. They don't make anything plain. Everything on the menu has some pretty exotic ingredients. It's also very pricy for burgers. This is the last place on Earth I would bring kids to eat. Burger King is right down the street.","date":"2011-10-27 14:32:57"}
{"review_id":"m1GlqFGIN5eayrbb2IbRZg","user_id":"B7YSV6r1ePAXc69FkDDuZw","business_id":"wZgUAuDuEGPEzKK-PsngKQ","stars":1.0,"useful":0,"funny":0,"cool":0,"text":"I wish I could give them zero stars. The call takers are rude. The technicians are incompetent. The manager doesn't know what customer service is. It is overpriced. I was quoted $230. I went to another locksmith, who actually got my lock off, and was quoted a total of $79.\n\nThe technician was late twice. He could not pick the lock which is not unusual and said he would have to drill through the lock. Ok. But, failed to bring charged batteries. He never offered to come back with charged batteries as soon as he could nor offered to send someone else. He said he could, however, come back around 10 hours later and do it \"under-the-table.\" Usually when you need to get a lock off, you need it off that minute. I said no. He charged me $30 for a service fee...a service that I never received. I contacted the manager, which seems to be a task in itself, and he said that I was rightfully charged $30 because I \"refused\" the technician's offer to come back later. I doubt the manager knew it was an under-the-table payment. I still have not been reimbursed my $30!","date":"2014-06-27 22:06:55"}
{"review_id":"j_eh6Iw7qseluR-7KfZOMg","user_id":"xpxWG7jQXZE6BcSeuIq4PQ","business_id":"mw_qxZJraNu7Q6u0GkcMew","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"This is a five-star restaurant if ever I have seen one! Maurice, one of the owners, met us at the front door, Daniel a seasoned waiter from Hawaii, took care of us and the food was over-the-top delicious. This is not just a restaurant, it is an experience!  My son recommended this to us and we are glad that we took his advice. I had the rack of lamb and my wife had the grouper with crab. Save room for dessert! We live in Tallahassee but the next time we are in Orlando it's back to Chatham's!","date":"2016-08-12 21:59:03"}
{"review_id":"egHMQBhFeaLhVyC6w3r0vQ","user_id":"HvpNr0ohHCaVLp014CQrdw","business_id":"gGvNgShksetPoimyKV8I9Q","stars":4.0,"useful":0,"funny":0,"cool":0,"text":"Quickly stopped in for a UFC fight. I sat down at the bar while my boyfriend parked the car. Although the restaurant\/bar was packed and LOUD, the bartender  (Vanessa) came over within a minute. Prompt service with a smile. When my boyfriend arrived a few minutes later, we didn't see Vanessa behind the bar so we tried to flag down her male coworker. He was caught up watching the fight and it took a few minutes to get his attention - which is fine. Not fine, however, is that after I ordered a beer, he stood and texted for a few minutes before getting my beer and held his arm up to shield his texting from management lol. \n\nVanessa, however, is an excellent bartender!! As loud as it is in this place, she continually scanned the bar looking for people attempting to make eye contact with her for service. I watched her work (my boyfriend and I have thirty years of food and beverage experience between the two of us) and she was focused and productive, but most importantly, she was ATTENTIVE! Give this girl a raise!!","date":"2015-10-04 05:37:01"}
{"review_id":"LBJJWJ0uNlIybMfPnQGS0A","user_id":"bUHweiErUJ36WGeNrPmEbA","business_id":"6Hm2FmfLcU_M91TrZI5htA","stars":5.0,"useful":5,"funny":3,"cool":3,"text":"I loved everything about this place. I've only been once but I keep meaning to go back as it was so great (just a bit out of my way). \n\nI went with a fairly large group so we'd all ordered something different (back when it was $6 for everything, it looks like they've changed that now). At the time I got the chickpea fries, which were delicious but watch out for the fiber because I had to sit out of an entire board game. The board game selection was great, and I love that they serve mead (though I didn't get any at 12 pm haha).  The molten lava chocolate cake was so, so good. Seriously.\n\nThe real topping to the experience (other than the great service and the fact that they put lemons in their complimentary ice water) was that they were playing Siouxsie and the Banshees in the background! That + board games + good food + mead = I'm a fan forever.","date":"2014-02-05 21:09:05"}
{"review_id":"ucFOnqgaV40oQ2YNyz5ddQ","user_id":"JHXQEayrDHOWGexs0dCviA","business_id":"KXCXaF5qimmtKKqnPc_LQA","stars":1.0,"useful":0,"funny":0,"cool":0,"text":"Great coffee and pastries. Baristas are excellent. All the other staff are sooooo lost and vacant.","date":"2018-03-03 23:45:25"}
{"review_id":"btNWW2kdJYfwpTDyzJO3Iw","user_id":"DECuRZwkUw8ELQZfNGef2Q","business_id":"zmZ3HkVCeZPBefJJxzdJ7A","stars":4.0,"useful":0,"funny":0,"cool":0,"text":"Nothing special but good enough.  I like another one much better in Dorchester, but hardly get to that area, so Le's is my goto place for Pho","date":"2012-12-04 04:29:47"}
{"review_id":"3c4LFWiVUHPDQzAd8uxc3A","user_id":"jySmPCkEkJR3cWJlkEs9cw","business_id":"ZW7aI5FO_3q_vSzI4_zx-Q","stars":5.0,"useful":5,"funny":2,"cool":4,"text":"Definitely 5 stars for the donuts.  Our family has been coming to Dandy Donuts for years!  Their donuts can't be beat and are a must-try.   The same Cambodian family has owned this shop for years. They are kind, hard-working folks that put out a consistently fresh and delicious line of donuts as well as croissants and apple fritters their customers love.   \n\nSubs are also good.   I notice they've added a new low-price lunch special of hot dog, chips and drink.  The surroundings are definitely \"hole-in-the-wall\", but go and try the donuts.","date":"2013-08-15 14:47:40"}
{"review_id":"GDgXjXSZCA1iNQWD7OHXfg","user_id":"1RCRKuHgP3FskGUVnmFdxg","business_id":"mOnesB4IF9j6-ZmHoOHOig","stars":4.0,"useful":0,"funny":0,"cool":0,"text":"I think their rice dishes are way better than their Pho's. Also I really like their sandwiches and spring rolls. I took a friend who haven't had viennese food ever. He absolutely love it.","date":"2017-05-26 03:05:46"}
{"review_id":"XjL87cCH-F7gVquxWKH7Sw","user_id":"ltn9yaWIarK_o4DeMT1duA","business_id":"MJYb0fioM-X7jc_0RIYyqQ","stars":4.0,"useful":0,"funny":0,"cool":0,"text":"Best station to fill up around in my opinion. Attendants are super friendly. They also carry bio diesel! Been running this in my Dodge for couple years w\/o any issues. With Safeway right on same corner, the rewards really make this a favorite gas stop. Thanks!","date":"2016-09-18 18:16:53"}
{"review_id":"1OsubwE6zKNU1fkBGxOFrQ","user_id":"vFnYYmtVwcMGyyGesNImVQ","business_id":"jMz_y_-cWMfiZF7Q5snE6Q","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"Ordered the original tonkotsu base ramen and a char siu don for my 2 year old.  Loved the soup base but especially loved the size of the bowl...unlike most ramen place, the size of the bowl reflected the quantity of the content in it.  Totally looking forward to going back so I can customize my next bowl of ramen...will select thicker noodle for sure and perhaps add more ramen!\nFabulous service and loved that they had a sign advising ppl to come in, write down name then wait outside, so not to clog the entrance\/exit.\n\nGreat ramen place, but won't go with a group larger than 4 ppl.  Heads up, there was only one high chair.","date":"2016-01-30 02:39:22"}
{"review_id":"OMGz2BaL3bzRvY2hEp1-wQ","user_id":"d01SZKYmReEar7varZB0HQ","business_id":"I2OblwJG8_XzFxxoHU0vVQ","stars":4.0,"useful":5,"funny":1,"cool":1,"text":"I just had my lasik done last week and I am really happy with my results. The staff were all extremely nice, remembered me like I was an old friend, and generally made me feel very comfortable. It's also great that they have Saturday appointments. The evaluation, as with most lasik places, was complementary and all followup exams and treatments are included.  These exams are generally about 10 minutes.\n\nThe pre-surgery paperwork and going over post-op details, etc took about 30 minutes and the surgery itself (both eyes) took 25 minutes. Numbing drops are applied, so there is no pain, but while the flap is being created, there is an intense feeling of pressure from the suction that holds your eye still. I hated that. Afterward, the only pain I experienced was similar to getting soap in my eye, a kind of stinging. I went home and slept all night and woke up the next morning for my first followup and I could see! And had no more burning! I had some broken blood vessels from the suction used to hold my eye in place, but no pain all week, just dry eyes. The week following, you must avoid swimming, exercise, eye makeup, and have to sleep with goggles (so you don't rub your eyes) and apply drops almost constantly. \n\nI believe I've made a great investment. I still can't believe I'm going to see this well FOREVER! :-) \n\nMy only complaint was that they didn't administer a sedative or anything to help me relax during the surgery (and maybe a pain Rx for after?) I didn't feel nervous before it started, but a little panic hit me when my eye was being held open and I was asked to keep my eye very still.\n\nOh, and they charge $2100 per eye for the all-laser lasik.","date":"2010-05-14 14:33:54"}
{"review_id":"-QpNdU_p44GR0NcRxDRyNQ","user_id":"ffJp-ZN80M4sSkDL8Ra18w","business_id":"WDGeeyeK7bG0cvq_ZglAdA","stars":1.0,"useful":0,"funny":0,"cool":0,"text":"Almost desolate restaurant and dingy evironment. That's a star.\n\nDecieving lunch menu. (claimed dishes were 550 before 3 on Saturdays and was still charged $2 more) That's a star.\n\nCongealed beef soup. That's a star.\n\nFriend's dish didn't come for 30 minutes after I got mine and he didn't get the soup that came with it. That's a star.\n\nAsked for water 5 times and didn't get of until right before the bill came. That's a star. \n\nWelp, I seem to be out of stars! Whoopsie Daisy! I wanted to be more forgiving since the waitress was new but even the manager who stepped in for her was screwing up. And it doesn't excuse the mediocre food.","date":"2009-03-01 01:05:50"}
{"review_id":"DSSqAIeB7IVNWZiF0COlSA","user_id":"7x863PREK98UuCxYpOszIw","business_id":"ISBs1ARjIFCXGWqPmGSMog","stars":5.0,"useful":5,"funny":0,"cool":0,"text":"I had a 1 hour massage tonight with Barbara. LITERALLY best massage I've had, and I've gotten plenty of massages around the globe. First off, let me say that when Barb asks you the simple questions, she follows through as you please. I hate when I request soft-medium and it's totally opposite. I received a free foot salt scrub due to a scheduling issue, not bad! When a company offers you something in addition due to a mistake on their part, take it! That's their way of apologizing indirectly! Awesome massage even without the add on! Cannot wait to book my next massage with her!","date":"2013-05-07 01:49:39"}
{"review_id":"WxrK52MRJy6Cxk_vOJeD1Q","user_id":"9d22a-5XXvsZ55p04gI5jA","business_id":"_gOz7-aHMyGUHOtjDrEv2w","stars":2.0,"useful":4,"funny":2,"cool":0,"text":"When we first arrived at the Coco Key, we were greeted by a number of staff members as we checked in, the process was slightly awkward because the staff appeared new or in training, which can happen from time to time. No harm no foul. \nThe foul part came when we entered our room. The carpet was soggy near the bathroom and the entire room was permeated with a musty smell. Something along the lines of aeu-de-someone-flooded-the-toilet. \nAfter unpacking--which I shouldn't have done, but I'm a sucker for any opportunity for sticking things out and making them work. We called the front desk. \nShortly thereafter, a housekeeper appeared with a spray bottle to investigate. She agreed that the carpet needed to be cleaned (which would happen the following day), and began spritzing, what I can only assume to be anti-mold stuff? I didn't ask--which clearly you can see from this point means I have some sort of timidness for communication. I really need to outgrow that. \nLong story short, by the time she had hosed down the carpet, walls, and doorways with her spray bottle, we were left coughing and choking on the fumes and were forced to spend the night with the door open in a weak attempt to ventilate the room.\nI did not leave my heart in FL. but I may have donated a small portion of lung tissue. \nYou're probably asking...\"but did you die?\" \nI am very much alive. However, I am very doubtful that I will ever return to the coco key.","date":"2017-09-20 15:38:27"}
{"review_id":"dprQ8G5J01mlV6Z5W19LtQ","user_id":"ednTYrjsoD3HtpZ-Z29SAQ","business_id":"AW_dMex_BXFzgBJFxAjDuQ","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"Erika was amazing! After months of trying to find a place on my own, she swooped right in and helped me find the perfect home in a couple of weeks! I couldn't be more excited or grateful!\n\nthanks Erika!","date":"2016-08-03 22:55:45"}
{"review_id":"g2xwd2o7ynGmdxhoDctooQ","user_id":"QpsbLV8dNFI9cxhrBGA0Rw","business_id":"tiKV9b0b2kpaGm5a-nd1ig","stars":4.0,"useful":0,"funny":0,"cool":0,"text":"This place is truly a hidden gem (literally hidden as you have to walk through a Chinese restaurant to get there).  The place is small, cozy, and charming inside.  And the food is better than what you might find in the north end.  The service is great and the servers do a great job at pronouncing the specials to you like a true Italian! Make sure you make a reservation as it does book up rather quickly.","date":"2015-01-03 21:34:06"}
{"review_id":"Cp5dqWk5XtMq8yk_Hlp-fw","user_id":"SIP1Dfwy1M0HWzdgGg9H9g","business_id":"I-qL4zwr62710OJBtVv28A","stars":4.0,"useful":0,"funny":0,"cool":0,"text":"Am I crazy for giving anything MBTA-related four stars? Maybe.\n\nRating the red line against other public transit services I've used in the US and abroad, no, it can't hold a candle to anything in Japan or some of the better Metro lines in Paris. But those lines are well-funded. That's simply not the case anywhere in the US.\n\nSimply because the T shuts down at night, it's cleaner than anything in NYC. Sure, it's not as fast, but Boston's not nearly as big as NYC, so it's fast enough.  Compared to any other public transit in the US, the red line is awesome. \n\nLook, all of the MBTA -- the trains, the busses, the stops -- they are all showing signs of neglect. It's in debt by several billion dollars. So they're not going to add any fancy new features or do any serious upgrading until they have money to pay for it.  So knowing all this, I'm still really quite happy with the red line.","date":"2008-04-29 17:33:18"}
{"review_id":"GgWFjRHhelaiUgR2-H6N4g","user_id":"pRPT3vqhqpU7kHgmKJamvw","business_id":"-_GnwXmzC3DXsHR9nyaC2g","stars":3.0,"useful":0,"funny":0,"cool":0,"text":"3.5 stars! I got the avocado margarita and it was delicious! Customer service was OK, but they were busy. The food was better than expected and the meat wasn't overly salty. I will definitely return!!!","date":"2012-11-06 07:09:57"}
{"review_id":"KKJEuhDDX6mFoMIbD-uqlw","user_id":"UwgDUYmon6Ob-5pVIYmzkQ","business_id":"CF2k7avLtV246iJwp16J7Q","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"First time being here and I'm not going anywhere else for my truck needs from now on.  Right when I feel like customer service is lacking in my day to day dealings, Miguel and the crew at Lambs turned all that around in a few minutes.  These guys got me in late in the day after work, gave me great advice about my truck without trying to pitch anything, took care of my tire quickly and had me on my way.  This place definitely stands out and I'm for sure coming back for an oil change, rotation, and whatever else I need in the future.  Thanks guys I really appreciate it.","date":"2017-06-27 23:10:58"}
{"review_id":"ldGCo22ax_ZvHV8TgE27Pg","user_id":"Df1kUPqooB1FrnwvXi5HsQ","business_id":"Qydynzc1ApxsSkVNEgPs9A","stars":1.0,"useful":0,"funny":1,"cool":0,"text":"I love ale house. I'm here all the time. The 1 star isn't for the place or for the service. The 1 star is for the management. You give absolutely zero respect to the game of soccer.\n\nI've been coming here since you opened and I've seen numerous times where you understaff for finals of tournaments where years past have shown you will be at capacity. \n\nIt's a shame you have a great establishment and great employees. If only management could sort it out this place would be 5 stars","date":"2014-07-13 18:59:44"}
{"review_id":"fzEqkhqC0P3feYtEx8vPlw","user_id":"FPafIu-ul1pFUCM2J-fP_A","business_id":"dWBKYjQ3q2v2dOjsfLLxDg","stars":1.0,"useful":1,"funny":0,"cool":0,"text":"This has not been a pleasant experience. According to our server, the restaurant is short on staff. This means our service has been frightfully slow. We have been here over an hour with receiving our food. Most of the food was good, except for the steak that was tough. This is a trendy place. Great spot to be seen. Food is just okay.","date":"2015-12-20 19:31:34"}
{"review_id":"oNNTEpc2PmB4w_vy9pbsJw","user_id":"s4NgvdIfBH3UQdccWCyARg","business_id":"IdXHHEUH4ebcxdRxCo3JNw","stars":3.0,"useful":7,"funny":4,"cool":1,"text":"\"A Bit Embarrassed\"\n\nI'm not exactly sure where to start with this update, other than saying that my intention was absolutely NOT to get my - or everyone's- meal comped last night.\n\nYes, I'm doing the Eat On $30 Challenge. Yes, I was swayed by another Yelper to go to Rosebud, even while fully aware that I didn't want the $5 Mystery Monday meal (biscuits and gravy with a fried egg - looked good when I saw James H.'s, Emily R.'s and Alain L.'s orders, but just not something I really like at all) so I was probably getting myself into a grossly-tempting land of foodiness, given all the other things I know or think sound goody good on their menu. I went anyway!I wanted the company and I'm so used to dining out, I couldn't resist, I suppose!!\n\nLong story short, the service was Capital-B Bad. BAD. Slow. Inattentive. As Lauren S. noted, no refills on water, plus other drinks were slow and things like James H.'s grits side and Emily R.'s fries were either forgotten (the former) or delayed and without steak sauce (the latter). Then, the first few folks to cash out clearly asked for separate checks, fiiiiiiinally got one that was all together, had to ask for it to be corrected . . . only to find that each one had a 20% \"service charge\" tacked on. Huh?! We all sat in the bar area, the group just kinda grew as people filtered in, and nowhere on the menu does it mention anything about an auto-add for \"big parties.\" Besides, I thought it was insulting because pretty much everyone at the table goes there a lot or at least often-enough to have talked with Ron and be a pretty familiar \"face\" - and we're Yelpers, so we're gonna tip you (oh, and PS? Not everyone ordered the $5 special, like Kathleen M. with her shrimp-n-grits or Lauren S. with her breakfast bowl . . . no one was gonna stiff them!).\n\nStill, our first three dining companions left without much fanfare, and I figured, \"hey, bad night.\" Then . . . the rest of us got ready to leave and a) our server took for-freakin'-ever to bring checks, and b) (the last straw in my book) dropped Emily R.'s to-go box on the floor, picked it up and resealed it, and dropped it in front of her like nothing had happened. Oh, did I mention that while we were waiting on our checks, we got to see several other servers (note: not ours) eating some of Ron's wings at the bar with \"regulars\" (as he later described them)?? Yeah, it was a BAD night at Rosebud.\n\nSo when our server finally came back with the checks, I spoke up. My intention was to let him know that we were \"nonplussed\" and, as I stated to him, he should remove the \"automatic 20% service charge if he'd included it.\" A couple minutes later, Ron came out to \"chat.\"\n\nI have to say . . . I'm \"a little bit embarrassed\" because I'm not the diner who wants her meal comped for any little reason, or, really, ever, unless it's a gracious gesture on management's part. And when Ron came over, it felt, well, like our server had tattled to Daddy . . . and that Ron didn't really want to hear what had happened. He said most of the right things, and \"agreed\" with comments, but suddenly I felt like I was the only one speaking up, he didn't *really* care, he noted he'd worked \"80 or 90 hours\" in the last week (ok, but you're the owner, right? The resto business is nothing close to easy . . . what does that have to do with our conversation??), and he tried to defend his server as well as justify the service charge . . . and then he said \"I can see where this is going . . . let me just take care of all of your dinners.\"\n\nWell, sh*t. Not really what I wanted to have happen. Honestly, it just seemed like an easy out to get *me* (us) to STHU. And then . . . how do I tip on a non-dinner? Especially one where I just had Bloody Mary's once noting how sloooow everyone's food orders came, despite being tempted? And do I just take it as a \"freebie\" given my week of Eat On $30 challengedom?? It was all too much. It seemed we all left awkwardly (not to mention that Kathleen, James, and Lauren had to pay, including a \"service charge\" and we didn't) and while Emily \"tipped for me\" since I didn't eat, but she knew I felt weird, I left with the overall thought . . . \"Huh?? What just happened??\"\n\nMonday. I will chalk it all up to Monday. I really like Rosebud and want to continue going often and recommending it to folks, but I left feeling -and still feel - like I need to *digest* my most recent experience . . .","date":"2009-10-13 22:20:10"}
{"review_id":"nJTSr-EGNhhA5o146THkPg","user_id":"T9O5pkKKlNvr-qqfefDlbA","business_id":"luOZQ9YBrWwP8mYrS4rNoA","stars":4.0,"useful":2,"funny":0,"cool":0,"text":"This place has some of the BEST chinese take-out in Portland. Don't let the inside-a-bar vibe scare you away. WE LOVE HAPPY FORTUNE. Just thinking about it makes me want to go there right now. Plus.. they have great prices. \n\nHere's some of our favorites over the years: General Tso's (nice and spicy), the crab puffs, lo-mein. \n\nThe service is great -- cheap drinks, while you're waiting, from plucky, yet-very-nice bartenders. Interesting people watching, but make sure to order from the bar-side of the place for that.","date":"2008-12-03 04:39:26"}
{"review_id":"ADPWjsySIpmuOSL07w7kSQ","user_id":"_soZ9DRjCF7Op7Us8bxSSA","business_id":"p2BkIrOuIsxGqtV0lwOZUw","stars":3.0,"useful":0,"funny":0,"cool":0,"text":"Just had Yalla for lunch today. I work on Krog Street, so figured I would give it a shot today. The line was VERY Long (about 10 people deep), which must mean it is good. I waited, and waited, and waited in line for about 20 minutes. My co-workers went to Fred's and had ordered and almost finished their food by the time I sat down at a table. I stuck it out in line though. The people behind the counter are very slow. Maybe because they have only been open for three weeks they are still working on the process. There is room for improvement on \"moving orders through the line.\"\n\nIt's basically like a burrito place, but Mediterranean food. I got the pita Shawarma ($8) which was a little pricey considering this was their smallest size. One size up is basically a Mediterranean tortilla which was $14. That is VERY steep vs a burrito place being about $6.50 or $7.\n\nThey did a great job loading it up though with lots of sauces and vegis. Food wise, I would give them a 4 as everything tasted great. Service wise, I would have to give them a 2 as the wait was terrible! 20 minutes in line is crazy for only having about 10 people in line. Good food, but the prices are very high too.","date":"2014-12-17 19:34:52"}
{"review_id":"7_Guwplx7k3vn-gtqLNFdg","user_id":"XBS1K_IzAya1uGpJwTk4GQ","business_id":"AvXqLbcGCxdIEF_qZTY0Kw","stars":5.0,"useful":1,"funny":1,"cool":0,"text":"Overall Recommendation:  If you like fried chicken, you should definitely check this place out.  Be forewarned that it's not a nice restaurant like the Flying Biscuit or Mary Macs so adjust your expectations for decor and service.  My husband and I are not from the South but as we ate our way through Atlanta, the Busy Bee Cafe ranked #1 on both our lists.\n\nPros\n  - BEST FRIED CHICKEN WE'VE EVER HAD...perfectly crispy skin with \n    great flavor. It seriously makes me not very excited to eat fried chicken \n    anytime soon because I know anywhere I go in Boston won't be as \n    good.\n  - Ribs with a nice, smokey taste\n  - Super mac n' cheese made with real cheese\n  - Dang good corn muffins you can munch on before your food arrives.\n\nCons\n  - Very busy so be prepared to wait (and you should because it's worth it)\n  - No air conditioning so in the summer it can get pretty toasty.  We were \n    there at the end of June and even with the fans going, it was warm.\n  - Sweet tea is too sweet for me\n\n**Note: There's a small parking lot as well as street parking.  However if you park on the street, the meters are really expensive...something crazy like $2 an hour.","date":"2011-06-29 01:26:18"}
{"review_id":"HgseF4yuseUvcrmCJXZliA","user_id":"hcItyz0modqnxavedxxNBQ","business_id":"xPo2OA5_wQgUEzAr-1o1Lg","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"best pizza on a busy street! my kids love it. i've been recommending to everyone who has a birthday party in that area to order from pizzapalooza and they are not disappointed. the pizza is thick, square and yummY! so many varieties, i love their pasta with meatball too! They also serve ice cream. This is a good value comfort food place for me and my family.","date":"2008-12-03 20:41:59"}
{"review_id":"vwPVD6AfV3Hi5F2bjKZK1w","user_id":"g-1DDTwuJArYg2ZHolRAOw","business_id":"vRrDTIW9IFBO4cc3laazUw","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"So sad to learn they are demolishing this. Always loved coming here and seeing all the graffiti art work.","date":"2018-03-15 20:34:25"}
{"review_id":"KeyNH8Jj05BZGtqjgCFnTg","user_id":"M20pTzUOo2yD5Cuh5shjSA","business_id":"M4jkJHewQXZvDV34Tuon8g","stars":1.0,"useful":0,"funny":0,"cool":0,"text":"Had the worst experience there today when I tried to have a breakfast. The service was slow waited at least 30 min for my French toast and when it got there I pulled a long grosse hair out of it. I'm never going there again!","date":"2014-08-10 20:40:15"}
{"review_id":"yXKu-60gP_378PX0xzyHHg","user_id":"uUrXZ2guG27PQUeR6u8K3w","business_id":"WtDOs3a6k_oPJmwiDh4qBQ","stars":2.0,"useful":3,"funny":1,"cool":2,"text":"I wanted this to be a great place, but I wasn't impressed.  I went there for dinner on a Monday.  I tried the pulled pork sandwich--it seemed like a sort of specialty.  My husband had the burger.  The buns left something to be desired.  The fries were okay but a little cold.  But at about $10 per entree for bar food, it wasn't as good as it should have been.  Service was fine--they were overstaffed.  I don't think I will be back anytime soon.","date":"2009-02-28 22:47:35"}
{"review_id":"lJ7rzbvT-l8KO8lHfEsXsg","user_id":"LV1ME-ibA2h0IGyFUUWhaQ","business_id":"H_RM2u1WWGU1HkKZrYq2Ow","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"Incredible donuts. Sometimes you have to go early to have the full selection but the donuts never disappoint.","date":"2017-08-07 19:34:13"}
{"review_id":"k_cSVNRR_VkRvoJvODhclg","user_id":"whAYipnvcovnm4CXYuo9cw","business_id":"zWp3VbtS1p6PEdOxImpL1A","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"New owner is raising the bar with designer womens clothing.","date":"2013-12-31 16:53:20"}
{"review_id":"Lyjp1HlZbV7w7DLfXO-RiQ","user_id":"I4cfBoRWEhB90HBdjh8z5Q","business_id":"Ld88I1LFRlxnFlQVIoHAJw","stars":5.0,"useful":2,"funny":2,"cool":1,"text":"Kudos to AutoNation! We had an oil change done at another franchise and found our Toyota leaking oil the next day. We took it back to said business and they came up with all kinds of reasons that they weren't at fault. We brought the car to AutoNation, and in less than an hour we were shown that the first place used the wrong type of oil filter. Bravo! faced with the evidence the first place gave us the refund we requested.\n\nThe indoor theater and popcorn are a nice touch too! :D","date":"2017-08-07 20:49:39"}
{"review_id":"m-9DK7NwYedIPj1RQ_sXdw","user_id":"JuM-lH05m6Ln8OPUTg8p0g","business_id":"H_RM2u1WWGU1HkKZrYq2Ow","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"Dont bother going to voodoo, just come here instead. They have such unique flavors and everything is so fresh. Oh, and try their cold brew too!","date":"2016-09-18 17:06:01"}
{"review_id":"gfwRzTeqcgzyKvFnFwU3TQ","user_id":"Sy_D4Tt_XJXSb03UigFZTA","business_id":"EuiU19EjhybpGLXqBzntgw","stars":1.0,"useful":4,"funny":0,"cool":0,"text":"Bummer. I was very disappointed.  So I finally stopped in for take out.  I got the fried haddock sandwich with fries.  The fries were frozen and greasy..  The haddock sandwich was not good.  Tasted  very fishy and frozen.  I had such high hopes their take out would be good.","date":"2012-04-28 11:39:17"}
{"review_id":"P320Yt8vFD3yjI34hq-umg","user_id":"IMfkbLVt_GJfD7zJ9T4fcw","business_id":"VPqWLp9kMiZEbctCebIZUA","stars":3.0,"useful":0,"funny":0,"cool":0,"text":"Overall is good, is worth the line up I don't think so, mainly Instagram addict's go here. Believe me the food is great, service is ok, the hype about it I still dont get it.","date":"2018-09-25 03:22:50"}
{"review_id":"U3hsChq7fgOkXpmO2kT40Q","user_id":"XpmQ4Ik5YxckGRfBxeekZA","business_id":"J7KvI94QUxab29LLeG6bew","stars":5.0,"useful":0,"funny":0,"cool":0,"text":"It's a pleasure doing business with them. Very good service and hospitable staff.","date":"2012-07-16 00:34:36"}
{"review_id":"EO5rALvJMkK8QEvUNs1gxg","user_id":"u2xPfv6_wcKt-lW-C1cV8A","business_id":"9P-lp3AWDXGayDqJz9VPwQ","stars":2.0,"useful":0,"funny":0,"cool":0,"text":"The ramen here is less than great. It came out luke warm and oily. The service we received was NOT good. I will not be returning.","date":"2018-02-11 03:30:12"}
'''

In [38]:
reviews = []
for x in s.split("review_id")[1:]:
    reviews.append(x.split('''"text":"''')[-1].split('''","date"''')[0])

In [39]:
i = set()
while len(i)<50:
    i.add(random.choice(Orders["Order_id"]))
len(i)

50

In [40]:
Comments["Order_id"] = list(i)
Comments["Cust_id"] = Orders["Cust_id"].iloc[Comments["Order_id"]-1].reset_index(drop=True)
Comments["Restaurant_id"] = Orders["Restaurant_id"].iloc[Comments["Order_id"]-1].reset_index(drop=True)
Comments["Driver_id"] = Orders["Driver_id"].iloc[Comments["Order_id"]-1].reset_index(drop=True)
Comments["Rest_Rating"] = [round(random.random()*3+2, 1) for i in range(50)]
Comments["Driver_Rating"] = [round(random.random()*3+2, 1) for i in range(50)]
Comments["Comments"]=reviews
Comments["Score_Rating"] = [round(random.random()*3+2, 1) for i in range(50)]
Comments["Comments"]=Comments["Comments"].str.replace("'" ,r"\'")

In [41]:
Comments

Order_id  Cust_id  Restaurant_id  Driver_id  Rest_Rating  Driver_Rating  \
0        130       80             17         19          4.7            3.8   
1          5       79             10         25          2.9            2.6   
2          9       72             12          4          4.5            2.6   
3         10       28             16         49          2.0            2.0   
4         11      129              9          4          3.2            2.9   
5        151       76              7         13          2.0            3.5   
6        154      116             10         32          4.0            2.5   
7        155      117             13         40          3.5            3.7   
8        156      196              6         43          2.5            2.7   
9        157       23             11         22          4.0            2.5   
10       158       25             18          9          3.7            4.7   
11       159        1             20          5          3.6            3.4   
12       160      139             15          6          4.1            5.0   
13        29       37              2         45          4.9            2.3   
14       162      122             14          2          2.2            4.3   
15        34      115             15         20          4.7            2.0   
16        36      171              8         27          3.8            4.0   
17       165       42             18         49          4.9            2.7   
18       164        1             19          1          4.0            4.2   
19        40       80             17         42          2.2            3.2   
20       172      182              4         36          3.2            2.7   
21       174      161             15          3          2.6            2.6   
22        47       71             10         15          3.7            2.5   
23       177      149             15         29          2.5            4.9   
24        52       40              3         50          2.1            4.4   
25       183      115             17         12          3.6            4.6   
26        58        3             19         44          4.6            3.8   
27       187       23              4          2          2.5            4.8   
28       188      154             13         15          3.4            2.6   
29       191      178             15         20          2.5            2.1   
30       192      147             10          4          2.2            3.0   
31       194      167             14         29          4.7            2.1   
32       195       60             13          7          2.3            4.5   
33       196       69             11          2          3.5            3.4   
34       197       55             10         10          2.8            4.0   
35       199       85             14         47          3.3            2.2   
36       200      133             17         23          4.6            3.3   
37        75       89             10         45          3.4            3.9   
38        78      118              4         27          2.2            4.9   
39        86      122              8         39          4.8            3.6   
40        88       73              9         42          4.0            4.5   
41        89      125              2         16          2.7            4.8   
42        94       59              7         17          4.2            4.1   
43       107       75              4         30          3.5            4.4   
44       108      102              6         45          2.4            3.0   
45       112      197              3         12          3.5            4.6   
46       115      162             19         26          2.6            2.2   
47       116       65             13         11          2.7            3.1   
48       121      152             10         46          2.8            2.7   
49       125      148              7         46       

In [42]:
for i, row in Comments.iterrows():
    s = f'''INSERT INTO `Comments` VALUES({row["Order_id"]}, {row["Cust_id"]}, {row["Restaurant_id"]}, {row["Driver_id"]}, {row["Rest_Rating"]}, {row["Driver_Rating"]}, '{row["Comments"]}', {row["Score_Rating"]})'''
    if i %10==0: print(i)
    cursor.execute(s)

0
10
20
30
40


In [43]:
db.commit()

# Save as CSV

In [45]:
User_data.to_csv("Customers.csv", index=False)

In [46]:
Restaurants.to_csv("Restaurants.csv", index=False)

In [47]:
Drivers.to_csv("Drivers.csv", index=False)

In [48]:
Customer_Address.to_csv("Customer_Address.csv", index=False)

In [49]:
Orders.to_csv("Orders.csv", index=False)

In [50]:
Delivered_By.to_csv("Delivered_By.csv", index=False)

In [51]:
Comments.to_csv("Comments.csv", index=False)